In [19]:
#!pip install gym

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('data\SPFB.RTS_090601_190813.txt',sep=',')
df['Timestamp'] = df["<DATE>"].astype(str) + df["<TIME>"].astype(str)
df['Timestamp'] = pd.to_datetime(df['Timestamp'],format='%Y%m%d%H%M%S')
df.set_index('Timestamp',inplace=True)
df.drop(['<DATE>','<TIME>'],axis=1,inplace=True)
df.columns = ['Open','High','Low','Close','Vol']
df.drop('Vol',axis=1,inplace=True)
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
conversion = {'Open' : 'first', 'High' : 'max', 'Low' : 'min', 'Close' : 'last'}
df=df.resample('30Min', how=conversion)
df.dropna(inplace=True)

C:\ProgramData\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  


In [4]:
def bar_norm(df):
    t2=df.copy()
    temp=df.copy()
    temp['dif_close_1_cum']=(temp['Close']-temp['Close'].shift()).shift().cumsum()
    t2['Open_std']=t2['Open'] - temp['dif_close_1_cum']
    t2['High_std']=t2['High'] - temp['dif_close_1_cum']
    t2['Low_std']=t2['Low'] - temp['dif_close_1_cum']
    t2['Close_std']=t2['Close'] - temp['dif_close_1_cum']
    t2['Open_std']=t2['Open_std'] - temp['Close'][0]
    t2['High_std']=t2['High_std'] - temp['Close'][0]
    t2['Low_std']=t2['Low_std'] - temp['Close'][0]
    t2['Close_std']=t2['Close_std'] - temp['Close'][0]
    #t2=t2-temp['Close'][0]
    t2=t2.iloc[2:]
    return t2

def bar_norm_all(df):
    df=bar_norm(df)
    df['std']=df['Close_std'].rolling(1250).std()
    df.dropna(inplace=True)
    df['Open_std']=df['Open_std']/df['std']
    df['High_std']=df['High_std']/df['std']
    df['Low_std']=df['Low_std']/df['std']
    df['Close_std']=df['Close_std']/df['std']
    df.drop('std',axis=1,inplace=True)
    return df
df=bar_norm_all(df)

In [5]:
df=df[-70000:]
df_train=df[:-30000]
df_valid=df[-30000:-15000]
df_test=df[-15000:]

In [6]:
import gym
import gym_RTStrade

In [7]:
env = gym.make('Trade-v1')
env.data_init(df_train,df_valid,df_test,df_test.iloc[:,:4])
obs=env.reset()

v1.2


In [9]:
import keras.layers as layers
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K
from keras.initializers import glorot_uniform


def get_policy_model(lr,dimen=(40,4)):
    inp = layers.Input(shape=(40,4),name="input_x")
    adv = layers.Input(shape=[1], name="advantages")
    
    x = layers.Conv1D (16,3,activation='relu')(inp)
    x = layers.Conv1D (24,3,activation='relu')(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D (32,3,activation='relu')(x)
    x = layers.Conv1D (48,3,activation='relu')(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Conv1D (64,3, activation='relu')(x)
    x = layers.Conv1D (96,3, activation='relu')(x)
    x = layers.MaxPool1D(2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(30,activation='relu')(x)
    out = layers.Dense(3,activation='softmax')(x)
    

    def custom_loss(y_true, y_pred):
        log_lik = K.log(y_true * (y_true - y_pred) + (1 - y_true) * (y_true + y_pred) + K.epsilon())
        return K.mean(log_lik * adv, keepdims=True)
        
    model_train = Model(inputs=[inp, adv], outputs=out)
    model_train.compile(loss=custom_loss, optimizer=Adam(lr))
    model_predict = Model(inputs=[inp], outputs=out)
    return model_train, model_predict

Using TensorFlow backend.


In [8]:
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
def discount_rewards(r, gamma=0.8):
    prior = 0
    out = []
    for val in r[::-1]:
        new_val = val + prior * gamma
        out.append(new_val)
        prior = new_val
    return np.array(out[::-1])

In [11]:
def score_train_model(model):   
    obs = env.reset_to_trainScore()
    obs_v=obs.values
    while True:
        if render:
            env.render()        
        state = np.reshape(obs_v, [1, 40, 4])
        predict = model.predict([state])[0]
        action = np.argmax(predict)
        obs, reward, done, _ = env.step(action)
        obs_v=obs.values
        if done:
            print('done')
            break
    
    return (env.net,env.action_count)

def score_valid_model(model):
    obs = env.reset_to_validScore()
    obs_v=obs.values
    while True:
        if render:
            env.render()        
        state = np.reshape(obs_v, [1, 40, 4])
        predict = model.predict([state])[0]
        action = np.argmax(predict)
        obs, reward, done, _ = env.step(action)
        obs_v=obs.values
        if done:
            print('done')
            break
    return (env.net,env.action_count)

def score_test_model(model):
    obs = env.reset_to_testScore()
    obs_v=obs.values
    while True:
        if render:
            env.render()        
        state = np.reshape(obs_v, [1, 40, 4])
        predict = model.predict([state])[0]
        action = np.argmax(predict)
        obs, reward, done, _ = env.step(action)
        obs_v=obs.values
        if done:
            print('done')
            break
    return (env.net,env.action_count)

def score_testReal_model(model):
    obs = env.reset_to_testReal()
    obs = bar_norm_all(obs)
    obs=obs.iloc[-40:,4:]
    obs_v=obs.values
    while True:
        if render:
            env.render()        
        state = np.reshape(obs_v, [1, 40, 4])
        predict = model.predict([state])[0]
        action = np.argmax(predict)
        obs, reward, done, _ = env.step(action)
        obs = bar_norm_all(obs)
        obs=obs.iloc[-40:,4:]
        obs_v=obs.values
        if done:
            print('done')
            break
    return (env.net,env.action_count)

In [12]:

gamma = 0
dimen = (40,4)
print_every = 300
batch_size = 100
num_episodes = 15000
render = False
lr = 1e-3
print_every_train = print_every*5

In [13]:
model_train, model_predict = get_policy_model(lr,dimen)
model_predict.summary()

W0113 17:08:38.530238 13424 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0113 17:08:38.546268 13424 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0113 17:08:38.547272 13424 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0113 17:08:38.580261 13424 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_x (InputLayer)         (None, 40, 4)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 38, 16)            208       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 36, 24)            1176      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 18, 24)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 16, 32)            2336      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 14, 48)            4656      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 48)             0         
__________

In [ ]:
with tf.device('/CPU:0'):
    env.data_init(df_train,df_valid,df_test,df_test.iloc[:,:4])
    obs=env.reset()
    obs_v=obs.values

    reward_sum = 0
    num_actions = 3

    states = np.empty(0).reshape(0,40,4)
    actions = np.empty(0).reshape(0,1)
    rewards = np.empty(0).reshape(0,1)
    discounted_rewards = np.empty(0).reshape(0,1)

    num_episode = 0
    losses = []

    while num_episode < num_episodes:
        state = np.reshape(obs_v, [1, 40,4])
        predict = model_predict.predict([state])[0]
        action = np.random.choice(range(num_actions),p=predict)

        states = np.vstack([states, state])
        actions = np.vstack([actions, action])

        obs, reward, done, _ = env.step(action)
        obs_v=obs.values
        reward_sum += reward
        rewards = np.vstack([rewards, reward])

        if done:
            discounted_rewards_episode = discount_rewards(rewards, gamma)       
            discounted_rewards = np.vstack([discounted_rewards, discounted_rewards_episode])

            rewards = np.empty(0).reshape(0,1)

            if (num_episode + 1) % batch_size == 0:
                rmax=max( abs(discounted_rewards.min()),abs(discounted_rewards.max()))
                discounted_rewards /= rmax
                discounted_rewards = discounted_rewards.squeeze()
                actions = actions.squeeze().astype(int)

                actions_train = np.zeros([len(actions), num_actions])
                actions_train[np.arange(len(actions)), actions] = 1

                loss = model_train.train_on_batch([states, discounted_rewards], actions_train)
                losses.append(loss)

                states = np.empty(0).reshape(0,40,4)
                actions = np.empty(0).reshape(0,1)
                discounted_rewards = np.empty(0).reshape(0,1)


            if (num_episode + 1) % print_every == 0:
                # Print status
                score_train,tact_train = (0,np.array([0,0,0]))
                if (num_episode + 1) % print_every_train == 0:
                    score_train,tact_train = score_train_model(model_predict)
                score_valid,tact_valid = score_valid_model(model_predict)
                score_test,tact_test = score_test_model(model_predict)

                print("Average reward for training episode {}: {:0.2f} Train Score: {:0.2f} ;{},{},{} Loss: {:0.6f} Valid Score: {:0.2f} ;{},{},{} TEST Score: \33[47m \33[37m  {:0.2f} ;{},{},{}  \033[0m ".format(
                (num_episode + 1), reward_sum/print_every, 
                score_train,tact_train[0],tact_train[1],tact_train[2],

                np.mean(losses[-print_every:]),
                score_valid,tact_valid[0],tact_valid[1],tact_valid[2],
                score_test,tact_test[0],tact_test[1],tact_test[2],

                ))

                model_predict.save('models/model_v1big_{}'.format(num_episode))
                reward_sum = 0

            num_episode += 1
            obs=env.reset()
            obs_v=obs.values



W0113 17:18:32.924329 13424 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras\backend\tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



done
done
Average reward for training episode 300: -545.14 Train Score: 0.00 ;0,0,0 Loss: 0.000467 Valid Score: -15029.00 ;0,14949,11 TEST Score:    -14792.00 ;0,14952,8   
done
done
Average reward for training episode 600: -977.77 Train Score: 0.00 ;0,0,0 Loss: 0.000698 Valid Score: -14960.00 ;0,14960,0 TEST Score:    -14960.00 ;0,14960,0   
done
done
Average reward for training episode 900: -697.89 Train Score: 0.00 ;0,0,0 Loss: 0.000707 Valid Score: -14960.00 ;0,14960,0 TEST Score:    -14960.00 ;0,14960,0   
done
done
Average reward for training episode 1200: -517.81 Train Score: 0.00 ;0,0,0 Loss: 0.000648 Valid Score: -14960.00 ;0,14960,0 TEST Score:    -14960.00 ;0,14960,0   
done
done
done
Average reward for training episode 1500: -687.34 Train Score: -39960.00 ;0,39960,0 Loss: 0.000637 Valid Score: -14960.00 ;0,14960,0 TEST Score:    -14960.00 ;0,14960,0   
done
done
Average reward for training episode 1800: -482.41 Train Score: 0.00 ;0,0,0 Loss: 0.000590 Valid Score: 802.00 ;0,

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.plot (np.mean(losses[-print_every:]))
plt.show